In [ ]:
!pip install pytube
import numpy as np

     |████████████████████████████████| 51kB 4.2MB/s 


In [ ]:
import json
import os
import pandas as pd
import csv
import keras
from numpy import array
import matplotlib.pyplot as plt
import subprocess
import cv2
from pytube import YouTube

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet import preprocess_input
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from keras import models
from keras import layers
from keras import optimizers
from keras import callbacks
from keras.layers import Dense, Activation, Dropout, Bidirectional
from keras.layers.recurrent import LSTM
from keras.models import Sequential
from tensorflow.keras.optimizers import SGD
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from imutils import paths
import matplotlib.pyplot as plt
import argparse
import pickle
import shutil
from keras.callbacks import ModelCheckpoint
from keras.optimizers import SGD
import sys
import ssl
import urllib.request
from sklearn.metrics import classification_report
from keras.utils import np_utils
from keras.preprocessing.image import img_to_array



In [ ]:
from google.colab import drive
drive.mount("/content/drive",force_remount=True)

Mounted at /content/drive


In [ ]:
def downloadVideo(link, name):
  try:
    yt = YouTube(link)
  except:
    print("connection error")
  video = yt.streams.get_highest_resolution()
  try:
    file = video.download()    
    os.rename(file, name+".mp4")
    print("renaming " + file + " to " + name + ".mp4")
  except:
    print("Error downloading ")
  print("finish downloading ", name)
  return

In [ ]:
import re
import math
from subprocess import check_call, PIPE, Popen
import shlex
re_metadata = re.compile('Duration: (\d{2}):(\d{2}):(\d{2})\.\d+,.*\n.* (\d+(\.\d+)?) fps')
def get_metadata(filename):
    '''
    Get video metadata using ffmpeg
    '''
    p1 = Popen(["ffmpeg", "-hide_banner", "-i", filename], stderr=PIPE, universal_newlines=True)
    output = p1.communicate()[1]
    matches = re_metadata.search(output)
    if matches:
        video_length = int(matches.group(1)) * 3600 + int(matches.group(2)) * 60 + int(matches.group(3))
        video_fps = float(matches.group(4))
        # print('video_length = {}\nvideo_fps = {}'.format(video_length, video_fps))
    else:
        raise Exception("Can't parse required metadata")
    return video_length, video_fps

def split_cut(filename, n, by, basePath):
    '''
    Split video by cutting and re-encoding: accurate but very slow
    Adding "-c copy" speed up the process but causes imprecise chunk durations
    Reference: https://stackoverflow.com/a/28884437/1862500
    '''
    assert n > 0
    assert by in ['size', 'count']
    split_size = n if by == 'size' else None
    split_count = n if by == 'count' else None
    
    # parse meta data
    video_length, video_fps = get_metadata(filename)
    # calculate split_count
    if split_size:
        split_count = math.ceil(video_length / split_size)
        if split_count == 1:        
            raise Exception("Video length is less than the target split_size.")    
    else: #split_count
        split_size = round(video_length / split_count)
    output = []
    for i in range(split_count):
        split_start = split_size * i
        pth, ext = filename.rsplit(".", 1)
        output_path = '{}-{}.{}'.format(pth, i+1, ext)
        cmd = 'ffmpeg -hide_banner -loglevel panic -ss {} -t {} -i "{}" -y "{}"'.format(
            split_start, 
            split_size, 
            filename, 
            output_path
        )
        # print(cmd)
        check_call(shlex.split(cmd), universal_newlines=True)
        output.append(output_path)
    return output

In [ ]:
from functools import partial


def load_config(configPath):    
  # save np.load
  np_load_old = partial(np.load)

  # modify the default parameters of np.load
  np.load = lambda *a,**k: np_load_old(*a, allow_pickle=True, **k)
  config = np.load(configPath).item()
  # restore np.load for future normal usage
  np.load = np_load_old
  print(config)
  num_input_tokens = config['num_input_tokens']
  nb_classes = config['nb_classes']
  labels = config['labels']
  expected_frames = config['expected_frames']
  labels_idx2word = dict([(idx, word) for word, idx in labels.items()])
  return num_input_tokens, nb_classes, labels, expected_frames, labels_idx2word

In [ ]:
def extractFeatures(model, video, outputPath):
    if os.path.exists(outputPath):
        return np.load(outputPath)
    count = 0
    vidcap = cv2.VideoCapture(video)
    success, image = vidcap.read()
    features = []
    while success:
        success, image = vidcap.read()
        if success:
            img = cv2.resize(image, (224, 224), interpolation=cv2.INTER_AREA)
            input = img_to_array(img)
            input = np.expand_dims(input, axis = 0)
            input = preprocess_input(input)
            feature = model.predict(input).ravel()
            features.append(feature)
            count += 1
    unscaled_features = np.array(features)
    np.save(outputPath, unscaled_features)
    return 

In [ ]:
import keras.backend as K
import gc
def scanExtractFeatrues(dataPath, outputPath):
    x_samples = []
    y_samples = []
    dirCount = 0
    model = None
    for f in os.listdir(dataPath):
        folderPath = os.path.join(dataPath, f)
        if not os.path.exists(outputPath):
            os.mkdir(outputPath)
        outputFolderPath = os.path.join(featurePath, f)
        if not os.path.exists(outputFolderPath):
          os.mkdir(outputFolderPath)
            
        dirCount += 1
        print("now working on ", f, " and start cleaning keras")
        if model is not None:
            del model
            K.clear_session()
            g = gc.collect()            
            print("Done Cleaning GPU")
        model = ResNet50(include_top = False, weights = "imagenet")
        model.compile(optimizer=SGD(), loss='categorical_crossentropy', metrics=['accuracy'])            
        for video in os.listdir(folderPath):                
            videoPath = os.path.join(folderPath, video)
            outputFile = outputFolderPath + os.path.sep + video.split(".")[0] + ".npy"
            extractFeatures(model, videoPath, outputFile)

    return 

In [ ]:
def processData(inputPath, outputPath, expected_frames):
  max_frames = 0
  counter = 0  
  frame_list = []
  num_input_tokens = None
  for f in os.listdir(inputPath):                
    samplePath = os.path.join(inputPath, f)
    for item in os.listdir(samplePath):
      itemPath = os.path.join(samplePath, item)
      print(itemPath)
      sample = np.load(itemPath)
      frames = sample.shape[0]
      if frames > expected_frames:
        sample = sample[0:expected_frames, :]
      elif frames < expected_frames:
        temp = np.zeros(shape=(expected_frames, sample.shape[1]))
        temp[0: frames, :] = sample
        sample = temp
      outputFilePath = os.path.join(outputPath, item)
      counter += 1
      if os.path.isdir(outputPath) == False:
        os.mkdir(outputPath)
      np.save(outputFilePath, sample)
    print("done processing data, with a total of ", counter , " files")
    return

In [ ]:
import gc
def load_model(modelPath, expected_frames, num_input_tokens, nb_classes):
  # first need to clean up keras
  keras.backend.clear_session()
  gc.collect()
  # now load the LSTM model
  model = Sequential()
  model.add(Bidirectional(LSTM(units=512, return_sequences=True), input_shape=(expected_frames, num_input_tokens)))
  model.add(Bidirectional(LSTM(10)))
  model.add(Dense(512, activation='relu'))
  model.add(Dropout(0.5))
  model.add(Dense(nb_classes))
  model.add(Activation("softmax"))
  model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
  model.load_weights(modelPath)
  return model

In [ ]:
def moveFiles():
  for item in os.listdir("/content"):
    if len(item.split(".")) > 1:
      if len(item.split("-")) <= 1:
        continue
      folder = item.split(".")[0].split("-")[0]
      folderPath = os.path.join("/content/videos", folder)
      if os.path.isdir(folderPath) == False:
        os.mkdir(folderPath)
      oldDest = os.path.join("/content", item)
      newDest = os.path.join(folderPath, item)
      shutil.move(oldDest, newDest)

In [ ]:
!ls "/content/"

drive  sample_data


In [ ]:
# first read the video URL to extract the video ID
testURL = pd.read_excel("/content/drive/MyDrive/Colab Notebooks/Youtube/youtube.xlsx", header=0)
dataPath = "/content/videos"
featurePath = "/content/features"
processedDataPath = "/content/processedData"
configPath = "/content/drive/MyDrive/Colab Notebooks/UCF101/config.npy"
modelPath = "/content/drive/MyDrive/Colab Notebooks/UCF101/blstmModel.h5"
testInfo = {}
links = {}
timeLength = 1
for index, row in testURL.iterrows():
  vidoeID = row[1].split('?')[1][2:]
  testInfo[row[0]] = vidoeID
  links[row[0]] = row[1]

for videoName, link in links.items():
  dataPath = "/content/videos"
  featurePath = "/content/features"
  processedDataPath = "/content/processedData"
  print("videoName: ", videoName)
  print("link: ", link)  
  downloadVideo(link, videoName)
  videoPath = os.path.join("/content", videoName+".mp4")  
  if os.path.isdir(dataPath) == False:
    os.mkdir(dataPath)
  videoFolderPath = os.path.join(dataPath, videoName)
  if os.path.isdir(videoFolderPath) == False:
    os.mkdir(videoFolderPath)
  split_cut(videoPath, timeLength, "size", videoFolderPath)
  timeLine = {}

  moveFiles()

  # next we get the timeline for each file
  for subVideo in os.listdir(videoFolderPath):
    print(subVideo)
    name = subVideo.split(".")[0]
    counter = int(subVideo.split(".")[0].split("-")[-1])
    startTime = (counter-1) * timeLength
    maxTime = get_metadata(videoPath)[0]
    endTime = 0
    if startTime + timeLength > maxTime:
      endTime = maxTime
    else:
      endTime = startTime + timeLength    
    timeLine[name] = [startTime, endTime]

  # now we extract all the features
  if os.path.isdir(featurePath) == False:
    os.mkdir(featurePath)
  scanExtractFeatrues(dataPath, featurePath)
  num_input_tokens, nb_classes, labels, expected_frames, labels_idx2word = load_config(configPath)
  processData(featurePath, processedDataPath, expected_frames)

  # now we load the model
  model = load_model(modelPath, expected_frames, num_input_tokens, nb_classes)
  for data in os.listdir(processedDataPath):
    dataPath = os.path.join(processedDataPath, data)
    x = np.load(dataPath)
    name = data.split(".")[0]
    predicted_class = np.argmax(model.predict(np.array([x]))[0])
    predicted_label = labels_idx2word[predicted_class]  
    print(predicted_label)
    if predicted_label == "BrushingTeeth":
      print("detecting brushing teeth, time frame: ", timeLine[name])  

videoName:  test_01
link:  https://www.youtube.com/watch?v=IpFpgvVcb4I
renaming /content/Blow Dry Mistakes You May Be Making and HOW to Fix Them!.mp4 to test_01.mp4
finish downloading  test_01
test_01-151.mp4
test_01-275.mp4
test_01-164.mp4
test_01-236.mp4
test_01-17.mp4
test_01-455.mp4
test_01-447.mp4
test_01-251.mp4
test_01-405.mp4
test_01-412.mp4
test_01-576.mp4
test_01-540.mp4
test_01-366.mp4
test_01-283.mp4
test_01-67.mp4
test_01-373.mp4
test_01-138.mp4
test_01-270.mp4
test_01-318.mp4
test_01-280.mp4
test_01-175.mp4
test_01-11.mp4
test_01-568.mp4
test_01-68.mp4
test_01-491.mp4
test_01-272.mp4
test_01-69.mp4
test_01-54.mp4
test_01-505.mp4
test_01-434.mp4
test_01-410.mp4
test_01-153.mp4
test_01-429.mp4
test_01-204.mp4
test_01-467.mp4
test_01-66.mp4
test_01-401.mp4
test_01-292.mp4
test_01-516.mp4
test_01-309.mp4
test_01-311.mp4
test_01-176.mp4
test_01-548.mp4
test_01-293.mp4
test_01-15.mp4
test_01-101.mp4
test_01-529.mp4
test_01-171.mp4
test_01-269.mp4
test_01-485.mp4
test_01-361.mp4

In [ ]:
!ls "/content/features/test_01"

test_01-10.npy	test_01-16.npy	test_01-21.npy	test_01-27.npy	test_01-4.npy
test_01-11.npy	test_01-17.npy	test_01-22.npy	test_01-28.npy	test_01-5.npy
test_01-12.npy	test_01-18.npy	test_01-23.npy	test_01-29.npy	test_01-6.npy
test_01-13.npy	test_01-19.npy	test_01-24.npy	test_01-2.npy	test_01-7.npy
test_01-14.npy	test_01-1.npy	test_01-25.npy	test_01-30.npy	test_01-8.npy
test_01-15.npy	test_01-20.npy	test_01-26.npy	test_01-3.npy	test_01-9.npy


In [ ]:
get_metadata("/content/videos/test_01/test_01-16.mp4")

(2, 23.98)